<a href="https://colab.research.google.com/github/shyamal-anadkat/mood-n-movies/blob/master/wikipedia_movie_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/shyamal-anadkat/mood-n-movies/master/data/processed/IMDB_top_1000_clean.csv?token=GHSAT0AAAAAABQICNLJA7PV6UTCPS7KG2YUYRJDKXQ'
df = pd.read_csv(url)
display(df)

,title,title_href,mc_link,genre,certificate,votes,metascore,directors,stars,runtime,rating,introduction,produce_year
0,Spider-Man: No Way Home,/title/tt10872600/,https://www.metacritic.com/movie/spider-man-no...,"Action,Adventure,Fantasy",PG-13,475200,71.0,PG-13,148 min,148,8.7,"With Spider-Man's identity now revealed, Peter...",2021
1,Licorice Pizza,/title/tt11271038/,https://www.metacritic.com/movie/licorice-pizz...,"Comedy,Drama,Romance",R,39990,90.0,R,133 min,133,7.6,The story of Alana Kane and Gary Valentine gro...,2021
2,Dune,/title/tt1160419/,https://www.metacritic.com/movie/dune-part-one...,"Action,Adventure,Drama",PG-13,495468,74.0,PG-13,155 min,155,8.1,Feature adaptation of Frank Herbert's science ...,2021
3,CODA,/title/tt10366460/,https://www.metacritic.com/movie/coda-2021?fta...,"Comedy,Drama,Music",PG-13,53577,74.0,PG-13,111 min,111,8.1,As a CODA (Child of Deaf Adults) Ruby is the o...,2021
4,West Side Story,/title/tt3581652/,https://www.metacritic.com/movie/west-side-sto...,"Crime,Drama,Musical",PG-13,31979,85.0,PG-13,156 min,156,7.7,"An adaptation of the 1957 musical, West Side S...",2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,C.R.A.Z.Y.,/title/tt0401085/,https://www.metacritic.com/movie/crazy?ftag=MC...,"Comedy,Drama",Not Rated,32590,81.0,Not Rated,129 min,129,7.9,"A young French Canadian, one of five boys in a...",2005
659,Vera Drake,/title/tt0383694/,https://www.metacritic.com/movie/vera-drake?ft...,"Crime,Drama",R,25210,83.0,R,125 min,125,7.6,Abortionist Vera Drake finds her beliefs and p...,2004
660,The Muppet Christmas Carol,/title/tt0104940/,https://www.metacritic.com/movie/the-muppet-ch...,"Comedy,Drama,Family",G,57185,64.0,G,85 min,85,7.7,The Muppet characters tell their version of th...,1992
661,The Edge of Heaven,/title/tt0880502/,https://www.metacritic.com/movie/the-edge-of-h...,Drama,Not Rated,31833,85.0,Not Rated,122 min,122,7.8,A Turkish man travels to Istanbul to find the ...,2007


In [23]:
from operator import is_
# import required modules
from bs4 import BeautifulSoup
import requests
import bs4

def is_right_wikipg(href, imdb_soup):
  retVal = False
  imdb_link_elem = imdb_soup.select_one("a[href*=https://www.imdb.com/title/]")
  
  if imdb_link_elem:
    imdb_link = imdb_link_elem["href"]
    to_compare = f"https://www.imdb.com{href}"
    if imdb_link == to_compare:
      retVal = True
      
  return retVal
    
def sensible_title_caps(str, no_caps_list = ["in","the"]):
    words = []  
    for word in str.split():
        if word not in no_caps_list:
            word = word.capitalize()
        words.append(word)
    return " ".join(words)


def create_df_with_plots(imdb_df):
  movie_title = imdb_df[['title','title_href','mc_link', 'produce_year']].values
  #print(movie_title)
  sample = movie_title[0:5]
  #print(sample) 

  missing_link = 0

  # example link dune.. https://en.wikipedia.org/wiki/Dune_(2021_film)

  for title,href,mc_link,year in movie_title:
    # get URL
    wiki_title = title.replace(" ", "_").replace(".", "").strip()
    # todo: capitalize first letter only if not like a, in
    page = requests.get(f"https://en.wikipedia.org/wiki/{wiki_title}")
    #print(year)
      
    # scrape webpage
    soup = BeautifulSoup(page.content, 'html.parser')
    
    is_right_wikipage = False

    imdb_soup = BeautifulSoup(page.text)
    
    is_right_wikipage = is_right_wikipg(href, imdb_soup)
    if not is_right_wikipage:
      # get the wiki page with like the year and film patter
      # I wanna set the soup to that new page if I find it.  West_Side_Story_(2021_film)
      suffix = f'_({year}_film)'
      page = requests.get(f"https://en.wikipedia.org/wiki/{wiki_title}{suffix}")
      soup = BeautifulSoup(page.content, 'html.parser')
      imdb_soup = BeautifulSoup(page.text)

      is_right_wikipage = is_right_wikipg(href, imdb_soup)
      if not is_right_wikipage:
        suffix = f'_(film)'
        page = requests.get(f"https://en.wikipedia.org/wiki/{wiki_title}{suffix}")
        soup = BeautifulSoup(page.content, 'html.parser')
        imdb_soup = BeautifulSoup(page.text)
        mc_link = []
        mc_link.append(imdb_soup.select_one("a[href*=https://www.imdb.com/title/]"))
        is_right_wikipage = is_right_wikipg(href, imdb_soup)
        if not is_right_wikipage:
          missing_link +=1
          missing_movies = []
          missing_movies.append(wiki_title)
          #print(f"IMDB link not available for {wiki_title}")
      
    

    final_plot = ""
    if is_right_wikipage:

      # create title object
      title = soup.find(id="firstHeading").text
      title_movie = []
      title_movie.append(title)
      #print(title)

      plots = []
      plot = []
      plot_section = [soup.findAll('h2')[1]]
      for header in plot_section:
        for elem in header.next_siblings:
            if elem.name and elem.name.startswith('h2'):
                # stop at next header
                break
            if elem.name == 'p':
                plot.append(elem.get_text())
      plots.append(plot)
      #print(f"{plots}\n")

      plots_dic = {
           'title': title_movie,
           'plot': plots,
           'mc_link': mc_link
            }
      #print(plots_dic)
      plots_df = pd.DataFrame(plots_dic, columns=['title','plot','mc_link'])
      #display(plots_df.head())
      

    # TODO- hearsch create df with mc_link, final_plot, imdb_title (mc_link so we can join with other tables)
  #plots_df = pd.DataFrame(plots_dic)
  #print(missing_link)
  return plots_df
create_df_with_plots(df)

,title,plot,mc_link
0,The Circus (1928 film),"[At a circus midway, the penniless and hungry ...",https://www.metacritic.com/movie/the-circus?ft...
